# Using Chebyshev's inequality to detect accumulated precipitation data anomalies.
The basic idea is to gather accumulated precip data from stations near each other. Under the assumption that, stations near each other will produce similar data to each other, we can treat the accumulated precip data as a random variable with some distribution that is unknown to us. Then we can use Chebyshev's Inequality or the Markov inequality to get and upper bound for the probability that a certain data point belongs to this distribution. All we need for this is the *sample mean* and *sample variance* of our local distribution which is simple to compute.

I will break this down into steps:
1. Given a station that we think is an anomaly or not find all of its neighbors in a reasonable radius (More on choosing the radius later)
+ Compute the mean and variance of the stations neighbors excluding the station itself (More on this as well).
+ Compute the Chebyshev Inequality and obtain an upper bound for the probability of such a point.
+ Gather results based on their bound and interpret the results.

Some important things to remember:
- The distribution of a stations neighbors may also be contaminated. This is the nature of the beast. We will explore the impacts of this depending on how contaminated the distribution is.
- This method heavily relies on the assumption that observations geographically close to each other behave similarly.
- For now we will only consider the accumulated precip. It may be possible to extend this to several variables.

## Chebyshev's Inequality
Chebyshev's Inequality is used to obtain an upper bound for the probability that a certain number is outside a distance from the mean. More precisely, let $X$ be a random varaible with some distribution $D$. Let $\mu, \sigma^{2}$ be the mean and variance, respectively, of your data. Then 
$$
\begin{equation}
P(|X-\mu| \geq \epsilon) \leq \dfrac{\sigma^{2}}{\epsilon^{2}}=\delta
\end{equation}
$$
#### Our case:
In our case $x \geq 0$ and we will only consider $P(X-\mu \geq \epsilon) \leq P(|X-\mu| \geq \epsilon) \leq \dfrac{\sigma^{2}}{\epsilon^{2}}=\delta$. We can obtain $\epsilon$ by considering the concentration inequality,
$$
P(X \geq \alpha)=P(X-\mu \geq \alpha-\mu)=P(X-\mu \geq \epsilon) \leq \dfrac{\sigma^{2}}{\epsilon^{2}}=\delta, \quad \epsilon = \alpha-\mu
$$
where $\alpha$ is the point of interest. For $0<\epsilon<1$ the problem is uninteresting and we can intuitively conclude that alpha is not a bad data point because it is very close to the mean, UNLESS $\sigma > \epsilon$. If $\delta>1$ then it strongly suggests that $\alpha$ belongs to the distribution.

## Example dataset
We will use this strategy on some simple synthetic distributions.
#### Uniform(0,1)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(10)
x = np.random.uniform(0,1,15)
print(x)
mu = x.mean()
var = x.var()
for i in np.linspace(0,1,10):
    alpha = i
    epsilon = alpha-mu
    bound = var/(epsilon**2)

    print('Mu: {}, \nVar: {}, \nAlpha: {}, \nBound: {}\n'.format(mu,var,alpha,bound))

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981 0.68535982 0.95339335
 0.00394827 0.51219226 0.81262096]
Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.0, 
Bound: 0.43941086385513173

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.1111111111111111, 
Bound: 0.7515456722288489

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.2222222222222222, 
Bound: 1.5685408003993397

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.3333333333333333, 
Bound: 5.086296804356288

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.4444444444444444, 
Bound: 128.11391266612617

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.5555555555555556, 
Bound: 14.058410781164257

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.6666666666666666, 
Bound: 2.5867504267057058

Mu: 0.4720924834365427, 
Var: 0.09793207613751041, 
Alpha: 0.7777777777777777, 
Bound: 1.04803519

As we can see the edge cases of a Uniform(0,1) i.e. $x=0,1$ are the least likely to happen because the mean is in the middle, $0.47$. Also note that a bound greater than 1 tells you that it is very likely that the point in questions belongs to the distribution.

#### Normal(5,2)

In [2]:
x = np.random.normal(5,2,15)
print(x)
mu = x.mean()
var = x.var()
for i in np.linspace(0,19,20):
    alpha = i
    epsilon = alpha-mu
    bound = var/(epsilon**2)

    print('Mu: {}, \nVar: {}, \nAlpha: {}, \nBound: {}\n'.format(mu,var,alpha,bound))

[7.98114727 6.51273386 5.93928185 4.53207482 5.70731588 8.56740188
 4.29939757 3.02070079 3.73149167 8.79752444 8.99304558 2.76265694
 9.36065661 5.38093376 7.5233718 ]
Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 0.0, 
Bound: 0.1220773781362704

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 1.0, 
Bound: 0.1734662724717406

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 2.0, 
Bound: 0.2657251510830664

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 3.0, 
Bound: 0.45725603648037005

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 4.0, 
Bound: 0.9654148383518125

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 5.0, 
Bound: 3.227016563870733

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 6.0, 
Bound: 109.44065558487507

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 7.0, 
Bound: 7.485870052943021

Mu: 6.20731564822049, 
Var: 4.7037350808930425, 
Alpha: 8.0, 
Bound: 1.4636431303535955

Mu: 6.20731564822049, 
Var: 4.7

For normally distributed data the edge cases are even more unlikely. Now that we have a grasp on how the estimation behaves we can apply it to some real precip data. First we will just be observing the behavior, then we will define reasonable bounds to categorize the data points.

## Testing Chebyshev's on Real World Datasets
Using the SynopticData API we will look at real precipitation data to see how well our model preforms. We will consider some specific datasets that have outliers and some that do not.

In [3]:
import requests
import json
import pandas as pd
from datetime import datetime
from pandas.io.json import json_normalize

Some helper functions from the Mahalanobis method

In [4]:
def get_query_dictionary(query_arg):
    """Creates a query dictionary with all of the parameters included in the query string.\n
    e.g. {token : 'demotoken', stid : 'NHMU', etc}.

    :param query_arg: query string to parse into dictionary
    :type query_arg: str
    :return: dictionary with query parameters
    :rtype: dict
    """
    params = query_arg.split(sep='&')

    keys = [arg.split('=')[0] for arg in params[1:] if arg]
    values = [arg.split('=')[1] for arg in params[1:] if arg]

    _dict = dict(zip(keys, values))

    if 'accum_hours' in _dict:
        _dict['accum_hours'] = _dict['accum_hours'].split(',')

    return _dict

In [5]:
def make_dataframe(query_string):
    """Constructs a dataframe from the query passed, assumes that the query is for pmode totals or last.

    :param query_string: string of the api query to make the dataframe out of
    :type query_string: str
    :return: dataframe with accumulated precip columns
    :rtype: pandas.DataFrame
    """
    resp = requests.get(query_string)

    # Any errors in the query
    assert resp.status_code == 200, "Server Error or URL Error"
    values_dict = json.loads(resp.text)
    if int(values_dict['SUMMARY']['RESPONSE_CODE']) == 2:
        print(values_dict['SUMMARY']['RESPONSE_MESSAGE'])
        return pd.DataFrame()

    assert int(values_dict['SUMMARY']['RESPONSE_CODE']) == 1, values_dict['SUMMARY']['RESPONSE_MESSAGE'] + '\n' + \
                                                              query_string
    # Success!
    df = json_normalize(values_dict['STATION'])
    # Get the arguments of the API call e.g. end, start, pmode, etc.
    query_dictionary = get_query_dictionary(query_string)

    # pmode last and intervals is not supported.
    if query_dictionary['pmode'] == 'last':
        print('pmode: last is not supported please use pmode totals using a start and end time.')
        exit(1)

    elif query_dictionary['pmode'] == 'intervals':
        print('pmode: intervals is not supported please use pmode totals using a start and end time.')
        exit(1)

    # Totals
    elif query_dictionary['pmode'] == 'totals':
        # parse the API call's start and end and compute a timedelta
        start = datetime.strptime(query_dictionary['start'], '%Y%m%d%H%M')
        end = datetime.strptime(query_dictionary['end'], '%Y%m%d%H%M')
        delta = end.timestamp() - start.timestamp()
        
        # initialize new columns to be added to the dataframe
        new_col = np.full(df.shape[0], np.nan, dtype='float64')
        new_date_col = np.full(df.shape[0], 0, dtype='int')
        
        # Unwrap the nested data
        for i, row in df.iterrows():
            if len(row['OBSERVATIONS.precipitation']) > 0:
                _dict = row['OBSERVATIONS.precipitation'][0]
                new_date_col[i] = (int(_dict['last_report']) - int(_dict['first_report']))
                new_col[i] = _dict['total']
        
        # Make the new columns from the unwrapped data
        df['ACCUM_' + str(int(delta/86400)).strip() + '_DAYS'] = new_col
        df['EPOCH_TIMEDELTA'] = new_date_col
        
        # filter out an observations that are not long enough
        df = df[abs(df['EPOCH_TIMEDELTA'] - delta) < .1*delta]

    df = df.apply(pd.to_numeric, errors='ignore')
    return df


#### Now for the actual computation. 

First we will consider a normal precipitation dataset that likely doesn't have any precip anomalies. These datasets are determined by using trusted stations and looking at the data to make decisions on whether the data is considered okay or normal. The particular dataset that we will use is based out of San Antonio. 

Second we will consider a couple data sets from Dugway in Utah where two stations did not report any precip from January 2018 to today. It is possible that a station will not report any precip throughout the majority of the year

#### Normal dataset

In [6]:
# A api call to a 'normal' set of precip data. not normal as in distribution, but in colloquial terms. 
normal_ex = "http://api.mesowest.net/v2/stations/precip?&radius=30,-97.8,15&token=demotoken&units=english&timeformat=%s&pmode=totals&start=201809150000&end=201810150000&network=1,2,213,234,106123,225,187"

In [29]:
import os.path

normal_ex_df = pd.DataFrame()

file_exist = os.path.isfile('./normal_ex.csv')

if file_exist:
    normal_ex_df = pd.read_csv('./normal_ex.csv')
else:
    print("File not found. Re-acquiring data from API. Results may vary.")
    normal_ex_df = make_dataframe(query_string=normal_ex)
    normal_ex_df.to_csv('./normal_ex.csv', index=False)
    
normal_ex_df

,DISTANCE,ELEVATION,ELEV_DEM,ID,LATITUDE,LONGITUDE,MNET_ID,NAME,OBSERVATIONS.precipitation,PERIOD_OF_RECORD.end,PERIOD_OF_RECORD.start,RESTRICTED,STATE,STATUS,STID,TIMEZONE,ACCUM_30_DAYS,EPOCH_TIMEDELTA
0,14.54,486,488.8,4329,30.183040,-97.679870,1,Austin-Bergstrom International Airport,"[{'count': 977, 'first_report': '1536969180', ...",2018-10-14T16:15:00Z,2002-08-14T00:00:00Z,False,TX,ACTIVE,KAUS,America/Chicago,3.04,2592000
1,8.31,597,587.3,4805,29.893610,-97.864720,1,"San Marcos, San Marcos Municipal Airport","[{'count': 1190, 'first_report': '1536969360',...",2018-10-14T16:15:00Z,2002-08-14T00:00:00Z,False,TX,ACTIVE,KHYI,America/Chicago,2.93,2592000
2,12.99,722,790.7,39734,30.176667,-97.874167,2,SOUTH AUSTIN RAWS,"[{'count': 719, 'first_report': '1536967080', ...",2018-10-14T16:18:00Z,2013-05-16T00:00:00Z,False,TX,ACTIVE,AURT2,America/Chicago,5.39,2592000
3,13.72,877,908.8,41685,30.083400,-98.008100,187,Driftwood 4 SSE,"[{'count': 2880, 'first_report': '1536969300',...",2018-10-14T16:10:00Z,2013-11-09T00:00:00Z,False,TX,ACTIVE,DRCT2,America/Chicago,5.80,2592000
4,11.47,862,885.8,41686,30.141630,-97.900200,187,Manchaca 4 W,"[{'count': 2880, 'first_report': '1536969300',...",2018-10-14T16:10:00Z,2013-11-09T00:00:00Z,False,TX,ACTIVE,MTCT2,America/Chicago,5.01,2592000
5,13.94,482,482.3,41687,30.177320,-97.688960,187,Onion Creek at Hwy 183 - Austin,"[{'count': 2879, 'first_report': '1536969300',...",2018-10-14T16:10:00Z,2013-11-09T00:00:00Z,False,TX,ACTIVE,ATIT2,America/Chicago,3.38,2592000
6,13.13,526,541.3,41694,29.937330,-97.593000,187,Lockhart 6 NE,"[{'count': 2878, 'first_report': '1536969300',...",2018-10-14T16:10:00Z,2013-11-09T00:00:00Z,False,TX,ACTIVE,LRCT2,America/Chicago,2.12,2592000
7,13.17,864,869.4,63148,29.939053,-98.008392,2,SAN MARCOS-WEST,"[{'count': 721, 'first_report': '1536966000', ...",2018-10-14T16:00:00Z,2017-08-22T20:06:00Z,False,TX,ACTIVE,SRWT2,America/Chicago,4.68,2595600
8,1.23,650,656.2,63149,29.998242,-97.820453,2,KYLE-EAST,"[{'count': 721, 'first_report': '1536966000', ...",2018-10-14T16:00:00Z,2017-08-22T21:06:00Z,False,TX,ACTIVE,KRET2,America/Chicago,3.84,2595600
9,13.98,588,597.1,63375,30.194930,-97.737390,225,Teri Road near Nuckols Crossing,"[{'count': 2877, 'first_report': '1536968700',...",2018-10-14T16:15:00Z,2017-09-26T18:59:00Z,False,TX,ACTIVE,TNCT2,America/Chicago,4.00,2592900


In [ ]:
# build function to compute chebyshev for particular stations to illustrate the effectiveness.

The column of interest here is the ACCUM_30_DAYS, but the data will be preserved for reporting later. Another step we will take is to save this data so that our testing remains unchanged

#### Anomaly 

In [8]:
dugway_ex_dpg13 = "http://api.synopticlabs.org/v2/stations/precipitation?token=demotoken&radius=DPG13,15&pmode=totals&start=201803010000&end=201810140000&units=english"
dugway_ex_dpg04 = "http://api.synopticlabs.org/v2/stations/precipitation?token=demotoken&radius=DPG,15&pmode=totals&start=201803010000&end=201810140000&units=english"